# Wires från dgn till gdb
## fas och topp för sig
## littera som attribut

In [ ]:
# Läs settings
import json

#settings_file = r"Q:\Projekt\Data_2024\styrfiler\settings_SEKNNO.json"
#settings_file = r"Q:\Projekt\Data_2024\styrfiler\settings_SEVPLI.json"
settings_file = r"C:\Users\SE1K4H\Desktop\SVK-Analys-Filer\settings_SEKNNO.json" # WOrking on Elsas's computer
#settings_file = r"C:\SVK_utveckling\settings_SEVPLI.json"

with open(settings_file, 'r', encoding='utf-8') as file:
    settings = json.load(file)

#print(settings)
print(json.dumps(settings, indent=4))
print("körning av cell klar")

In [ ]:
import arcpy, os, glob
import pandas as pd
import shutil

run_ID = settings["run_ID"]
powerline_list = settings["powerline_list"]
local_dir = settings["local_folder"]
dgn_dir = settings["dgn_folder"]
wires_gdb_template = settings["wires_gdb_template"]
domains_folder = settings["domains_folder"]

cvd_LEDNINGSGATA_path = os.path.join(domains_folder, "cvd_LEDNINGSGATA.txt")
wires_gdb = os.path.join(local_dir, f"wires_{run_ID}.gdb")

# Create wire gdb as copy of template
if os.path.exists(wires_gdb):
    print(f"{wires_gdb} finns redan")
else: 
    shutil.copytree(wires_gdb_template, wires_gdb)

arcpy.env.overwriteOutput = True

def wires_to_gdb(row):
    # Ange i definitionen att row är en pandas serie
    LG = row["LG"]
    line = row["line"]
    littera = row["Littera"]
    LG_code = cvd_LEDNINGSGATA[LG]
    
    print(f"Påbörjar export av fas- och topplinor för {LG}_{line}, {littera}")
    
    dgn = os.path.join(dgn_dir, f"{LG}_{line}.dgn")
    arcpy.env.workspace = dgn_dir # Varför ska den ha dgn dir som workspace?
    
    # faslinor
    fc_fas = f"{LG}_{line}_fas"
    # Create a value table that will hold the input feature classes for Merge
    vTab_fas = arcpy.ValueTable()
    layer_fas = "Layer_fas"
    arcpy.Delete_management(layer_fas)
    arcpy.MakeFeatureLayer_management(dgn + '/Polyline', layer_fas, "\"Level\" = 1")
    vTab_fas.addRow(layer_fas)
        
    # topplinor
    fc_topp = f"{LG}_{line}_topp"
    # Create a value table that will hold the input feature classes for Merge
    vTab_topp = arcpy.ValueTable()
    layer_topp = "Layer_topp"
    arcpy.Delete_management(layer_topp)
    arcpy.MakeFeatureLayer_management(dgn + '/Polyline', layer_topp, "\"Level\" = 2")
    vTab_topp.addRow(layer_topp)
    
    # Merge the CAD features into one feature class
    arcpy.env.workspace = wires_gdb
    
    # faslinor
    arcpy.Merge_management(vTab_fas, fc_fas)
    # Add and populate littera field
    arcpy.management.AddField(fc_fas, "Littera", "TEXT")
    arcpy.management.CalculateField(fc_fas, "Littera", "".join(("'",littera,"'")))
    arcpy.management.AddField(fc_fas, "LG", "LONG")
    arcpy.management.CalculateField(fc_fas, "LG", int(LG_code))
    arcpy.management.AddField(fc_fas, "lintyp", "TEXT")
    arcpy.management.CalculateField(fc_fas, "lintyp", "'ledare'")
    # domains
    arcpy.management.AssignDomainToField(fc_fas, "LG", "cvd_LEDNINGSGATA")
    arcpy.management.AssignDomainToField(fc_fas, "Littera", "cvd_LITTERA_LEDNING")
    
    # ta bort fält som inte ska användas DETTA BEHÖVER GÖRAS PÅ BÅDE FAS OCH TOPP MEN NUVARANDE 
    # UPPREPNING ÄR KLUMPIG
    keep_fields = ["LG", "Littera", "lintyp", "Shape", "Shape_Length", "OBJECTID"]
    all_fields = arcpy.ListFields(fc_fas)
    fields_to_delete = [field.name for field in all_fields if field.name not in keep_fields]
    if fields_to_delete:
        arcpy.DeleteField_management(fc_fas, fields_to_delete)   

    # Räkna faslinor, varna om inga finns
    arcpy.management.DefineProjection(fc_fas, sr)
    result = arcpy.management.GetCount(fc_fas)
    count = int(result[0])
    if count == 0:
        print(f"VARNING: Inga faslinor hittades i level 1 i {LG}_{line}.dgn")
    
    # topplinor
    arcpy.Merge_management(vTab_topp, fc_topp)
    # Add and populate littera field
    arcpy.management.AddField(fc_topp, "Littera", "TEXT")
    arcpy.management.CalculateField(fc_topp, "Littera", "".join(("'",littera,"'")))
    arcpy.management.AddField(fc_topp, "LG", "LONG")
    arcpy.management.CalculateField(fc_topp, "LG", int(LG_code))
    arcpy.management.AddField(fc_topp, "lintyp", "TEXT")
    arcpy.management.CalculateField(fc_topp, "lintyp", "'topplina'")
    # domains
    arcpy.management.AssignDomainToField(fc_fas, "LG", "cvd_LEDNINGSGATA")
    arcpy.management.AssignDomainToField(fc_fas, "Littera", "cvd_LITTERA_LEDNING")
    
    # ta bort fält som inte ska användas
    keep_fields = ["LG", "Littera", "lintyp", "Shape", "Shape_Length", "OBJECTID"]
    all_fields = arcpy.ListFields(fc_topp)
    fields_to_delete = [field.name for field in all_fields if field.name not in keep_fields]
    if fields_to_delete:
        arcpy.DeleteField_management(fc_topp, fields_to_delete)
    
    arcpy.management.DefineProjection(fc_topp, sr)

    # Räkna topplinor, varna om inga finns
    result = arcpy.management.GetCount(fc_topp)
    count = int(result[0])
    if count == 0:
        print(f"VARNING: Inga topplinor hittades i level 2 i {LG}_{line}.dgn")
        
    # Städa bort tillfälliga lager från minnet
    arcpy.env.workspace = dgn_dir
    arcpy.Delete_management(layer_fas)
    arcpy.Delete_management(layer_topp)
        
# Sätt detta i en main-funktion för tydlighet?
sr = arcpy.SpatialReference("SWEREF99_TM", "RH2000")
powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
df_cvd_LEDNINGSGATA = pd.read_csv(cvd_LEDNINGSGATA_path, sep="\t", header=0)
cvd_LEDNINGSGATA = {df_cvd_LEDNINGSGATA.Description[i]: df_cvd_LEDNINGSGATA.Code[i] for i in range(len(df_cvd_LEDNINGSGATA))}

powerlines_df.apply(wires_to_gdb, axis=1)
print("körning av cell klar")

# Kontroll

In [ ]:
arcpy.env.workspace = wires_gdb

feature_classes = arcpy.ListFeatureClasses()
print(f"Antal featureklasser: {len(feature_classes)}")

for fc in feature_classes:
    feature_count = int(arcpy.management.GetCount(fc)[0])
    print(f"{fc}: {feature_count} features")
print("körning av cell klar")

In [ ]:
print(powerline_list)

# Convert wires gdb to PostGIS

## Load Settings from File

In [ ]:
import json
import os

settings_file = r"C:\Users\SE1K4H\Desktop\SVK-Analys-Filer\settings_SEKNNO.json" # Working on Elsa's computer

try:
    if not os.path.exists(settings_file):
        raise FileNotFoundError(f"Could not find {settings_file}")

    with open(settings_file, 'r', encoding='utf-8') as file:
        settings = json.load(file)

    run_ID = settings["run_ID"]
    powerline_list = settings["powerline_list"]
    local_dir = settings["local_folder"]
    dgn_dir = settings["dgn_folder"]
    wires_gdb_template = settings["wires_gdb_template"]
    domains_folder = settings["domains_folder"]
    powerlines_folder = settings["powerlines_folder"]
    module_path = settings["modules"] 
    ogr2ogr_path = settings["ogr2ogr_path"]   
    proj_lib_path = settings["proj_lib_path"]
    gdal_data_path = settings["gdal_data_path"]
    DEFAULT_DB_NAME = settings["default_db_name"]
    DB_NAME = settings["db_name"]
    USER = settings["db_user"]
    PASSWORD = settings["db_password"]
    HOST = settings["host"]
    PORT = settings["port"] 

    if None in (run_ID, powerline_list, local_dir, dgn_dir, wires_gdb_template, domains_folder, powerlines_folder, module_path, ogr2ogr_path, proj_lib_path, gdal_data_path, DEFAULT_DB_NAME, DB_NAME, USER, PASSWORD, HOST, PORT):
        raise KeyError(f"One or more keys are missing in {settings_file}")
    
    print(f"Settings loaded successfully.")
except json.JSONDecodeError as e:
    print(f"Error: Invalid JSON format in {settings_file}")
except KeyError as e:
    print(f"Error: {e}")
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Error: Unexpected error: {e}")

Settings loaded successfully.


## Load PostGIS methods

In [ ]:
import psycopg

# TODO can move these kind of methods to utils or similiar?
def connect_to_database(db_name):
    try:
        conn = psycopg.connect(dbname=db_name, user=USER, password=PASSWORD, host=HOST, port=PORT)
        conn.autocommit = True
        return conn
    except psycopg.Error as e:
        print(f"Error connecting to {db_name}: {e}")
        return None
    
def execute_query(conn, query, data=None):
    try:
        with conn.cursor() as cur:
            cur.execute(query, data or ())
    except psycopg.Error as e:
        print(f"Error executing query: {e}")

## Set up PostGIS Database

In [5]:
import os
import psycopg
from psycopg import sql

def create_database(cur):
    drop_database_query = sql.SQL("DROP DATABASE IF EXISTS {db_name}").format(db_name=sql.Identifier(DB_NAME))
    create_database_query = sql.SQL("CREATE DATABASE {db_name}").format(db_name=sql.Identifier(DB_NAME))

    cur.execute(drop_database_query)
    cur.execute(create_database_query)

try:  
    # Step 1: Create a new database
    conn_deafult = connect_to_database(DEFAULT_DB_NAME)
    execute_query(conn_deafult, sql.SQL("DROP DATABASE IF EXISTS {db_name}").format(db_name=sql.Identifier(DB_NAME)))
    execute_query(conn_deafult, sql.SQL("CREATE DATABASE {db_name}").format(db_name=sql.Identifier(DB_NAME)))    

    # Step 2: Enable PostGIS extension
    conn_db = connect_to_database(DB_NAME)
    execute_query(conn_db, sql.SQL("CREATE EXTENSION IF NOT EXISTS postgis;"))

    print(f"Database {DB_NAME} was created successfully.") #TODO Verify this as well, don't assume
except Exception as e:
    print("Error: ", e)
finally:
    if conn_deafult is not None:
        conn_deafult.close()
        print(f"Connection to database {DEFAULT_DB_NAME} closed.")
    
    if conn_db is not None:
        conn_db.close()
        print(f"Connection to database {DB_NAME} closed.")

Database test_postgis_db was created successfully.
Connection to database postgres closed.
Connection to database test_postgis_db closed.


## Convert wires gdb to PostGIS

In [6]:
import os
import subprocess
import pandas as pd

#TODO in general in all cells, take a look att exception handling
PG_CONNECTION = f"PG:host={HOST} dbname={DB_NAME} user={USER} password={PASSWORD} port={PORT}"

def convert_gdb_to_postGIS(row):
    LG = row["LG"]
    line = row["line"]  
    table_name = f"{LG}_{line}_fas"
    ogr_command = [
        ogr2ogr_path,
        "-f", "PostgreSQL",
        PG_CONNECTION,
        wires_gdb,
        "-a_srs", "EPSG:3006",
        "-nln", table_name,
        "-nlt", "MULTILINESTRINGZ",
        "-dim", "3",
        "-overwrite", #TODO change this when there are several dgn-files, eg. append
        table_name        
    ]

    try:
        os.environ["PROJ_LIB"] = proj_lib_path #TODO potentially solve this in another way. Now it has to be done in order to find -a_srs EPSG:3006
        os.environ["GDAL_DATA"] = gdal_data_path
        subprocess.run(ogr_command, check=True, capture_output=True, text=True)
        print(f"Successfully imported {wires_gdb} into PostGIS as '{table_name}'.") #TODO verify this
    except subprocess.CalledProcessError as e:
        print(f"Error: Could not import {wires_gdb} into PostGIS. {e}")

try:   
    wires_gdb = os.path.join(local_dir, f"wires_{run_ID}.gdb")
    powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
    powerlines_df.apply(convert_gdb_to_postGIS, axis=1)
except ValueError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

Successfully imported C:/Users/SE1K4H/Desktop/SVK-Analys-Filer/pythonkörningar/test_postGIS\wires_250201.gdb into PostGIS as 'LG001_1_fas'.
